In [93]:
%run gower.ipynb
from pymongo import MongoClient
from random import randint
import pandas as pd
import numpy as np
import re
import category_encoders as ce
from scipy.spatial.distance import pdist,squareform, jaccard, cosine
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

gower.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int6

In [2]:
client = MongoClient(port=27017)
db = client.weare    

In [14]:
student_queries = db.students.find({})
students = pd.DataFrame(list(student_queries))

In [19]:
students.Email

0       sxa459@psu.edu
1      CFD5112@PSU.EDU
2      MAG5188@PSU.EDU
3      ama5044@psu.edu
4       axg130@psu.edu
5       kub288@psu.edu
6      AMB5012@PSU.EDU
7      LQB5283@PSU.EDU
8      mjg5978@psu.edu
9      EKC5166@PSU.EDU
10        cke5@psu.edu
11       JPK14@PSU.EDU
12      gjd131@psu.edu
13     SMC1079@PSU.EDU
14     MMK5725@PSU.EDU
15        TPA3@PSU.EDU
16     smb6742@psu.edu
17      amb397@psu.edu
18        ndd6@psu.edu
19     JME5033@PSU.EDU
20     TMC5621@PSU.EDU
21      LDC144@PSU.EDU
22       rnh16@psu.edu
23     EJC5174@PSU.EDU
24      auc455@psu.edu
25       ahb13@psu.edu
26      JZC140@PSU.EDU
27     JAK5973@PSU.EDU
28     mrc5163@psu.edu
29     AXE5177@PSU.EDU
            ...       
351    sbr5275@psu.edu
352    smj5601@psu.edu
353    lcc5242@psu.edu
354    glb5210@psu.edu
355       cgg7@psu.edu
356    AMB7119@PSU.EDU
357      lzk33@psu.edu
358    AME5495@PSU.EDU
359      muf93@psu.edu
360    mve5185@psu.edu
361     mdj169@psu.edu
362    DTB5189@PSU.EDU
363     cpk

In [26]:
students['id']=students.Email.str.split("@", n=1, expand=True)[0]
if '_id' in students.columns:
    del students['_id']
students.id

0       sxa459
1      CFD5112
2      MAG5188
3      ama5044
4       axg130
5       kub288
6      AMB5012
7      LQB5283
8      mjg5978
9      EKC5166
10        cke5
11       JPK14
12      gjd131
13     SMC1079
14     MMK5725
15        TPA3
16     smb6742
17      amb397
18        ndd6
19     JME5033
20     TMC5621
21      LDC144
22       rnh16
23     EJC5174
24      auc455
25       ahb13
26      JZC140
27     JAK5973
28     mrc5163
29     AXE5177
        ...   
351    sbr5275
352    smj5601
353    lcc5242
354    glb5210
355       cgg7
356    AMB7119
357      lzk33
358    AME5495
359      muf93
360    mve5185
361     mdj169
362    DTB5189
363     cpk104
364    smr5903
365    CMC6307
366    hdk5064
367     gmf148
368     SWF127
369    NMB5560
370    AMB7229
371    VVJ5031
372    jlg6213
373    HDA5030
374    MZK5588
375    seb5771
376    JBD5273
377    KKD5199
378      jrc88
379    AOG5246
380    eve5094
Name: id, Length: 381, dtype: object

In [28]:
#drop rows with 5 or more null values
students.dropna(thresh=5, inplace=True)
students = students.replace('', np.nan)

In [37]:
#number of courses, credits are too dirty as many entered text instead of a number
drop_columns = ["surveyCompletion", "Duration (in seconds)", "NumCourses", "transferCredits", "NumTranCredits",
                "WhyProfile", "otherChannels", "otherEmploy", "NonUS", "otherIndusry", "KnownThroughProfile",
                "otherEth", "PPLinPerson", "otherEmail"]
#only drop these columns if they actually exist in the data
drop_columns = [col for col in drop_columns if col in students.columns]
students.drop(columns=drop_columns, inplace=True)
#names of all the soc columns
SOCs = ['SOC'+str(x+1) for x in range(10)]
#names of all the CCE columns
CCEs = ['CCE'+str(x+1) for x in range(24)]
#mapping from each text column to ordinal values
#format is array of dicts containing keys 'col' and 'mapping'
ordinal_cols_mapping = []

In [38]:
#the following are mappings from textual column values to numberical ones
important_scale = [
        ('Extremely important', 7),
        ('Pretty important', 6),
        ('Fairly important', 5),
        ('Moderately important', 4),
        ('Somewhat important', 3),
        ('A little important', 2),
        ('Not at all important', 1)
]

interest_scale = [
    ('Extremely interested', 5),
    ('Rather interested', 4),
    ('Somewhat interested', 3),
    ('A bit of interest', 2),
    ('Not at all interested', 1)
]

Agree_scale = [
        ('Strongly agree', 5),
        ('Somewhat agree', 4),
        ('Neither agree nor disagree', 3),
        ('Somewhat disagree', 2),
        ('Strongly disagree', 1)
]

In [40]:
#fill in the mapping array 
for SOC in SOCs:
    ordinal_cols_mapping.append({
        "col": SOC,
        "mapping": Agree_scale
    })

for CCE in CCEs:
    ordinal_cols_mapping.append({
        "col": CCE,
        "mapping": Agree_scale
    })

for PeerInfo in ["PeerAvail", "PeerProfession", "PeerEdu", "PeerDetails"]:
    ordinal_cols_mapping.append({
        "col": PeerInfo,
        "mapping": important_scale
    })

ordinal_cols_mapping.append({
    "col": "Mconnected",
    "mapping": interest_scale
})

In [43]:
encoder = ce.OrdinalEncoder(mapping=ordinal_cols_mapping, return_df=True)
df = encoder.fit_transform(students)
#Create new columns by taking the mean of related columns for each row
df['SOC_f'] = df.apply(lambda row: np.mean([row['SOC9'], row['SOC10']]), axis=1)
df['SOC_id'] = df.apply(lambda row: np.mean([row['SOC1'], row['SOC2'], row['SOC3'], row['SOC4'], row['SOC5']]), axis=1)
df['CCE_IR'] = df.apply(lambda row: np.mean([row['CCE13'], row['CCE10'], row['CCE19'], row['CCE9']]), axis=1)
df['CCE_Coor'] = df.apply(lambda row: np.mean([row['CCE24'], row['CCE20'], row['CCE22'], row['CCE21'], row['CCE17']]), axis=1)
df['CCE_SS'] = df.apply(lambda row: np.mean([row['CCE1'], row['CCE2'], row['CCE3'], row['CCE4']]), axis=1)
df.drop(columns=CCEs + SOCs, inplace=True)

In [44]:
none_i = re.compile(r'none', flags=re.IGNORECASE)
df['kids'].replace(none_i, 0, inplace=True)
none_i = re.compile(r'zero', flags=re.IGNORECASE)
df['kids'].replace(none_i, 0, inplace=True)
stringany = re.compile(r'[a-zA-Z ()]+', flags=re.IGNORECASE)
print(f'Before digit processing, kids are {df.kids.unique()}')
df.kids = df['kids'].replace(stringany, '')

Before digit processing, kids are [ 0.  1.  2.  3.  4. nan  5.  9.]


In [52]:
categorical_cols = ["gender", "InUS", "ethnicity", "Usstate", "marrital", "employment", "industry"]
categorical_cols = [c for c in categorical_cols if c in df.columns]
df_c_mode = df[categorical_cols].mode()
print(f'mode listed are {df_c_mode.iloc[0]}')
print(len(df_c_mode))
# df.loc[:, categorical_cols].fillna(df_c_mode.iloc[0], inplace=True)
for col in categorical_cols+['kids']:
    df[col].fillna(df[col].mode().iloc[0], inplace=True)
onehotecoder = ce.OneHotEncoder(cols=categorical_cols, handle_unknown='ignore')
df = onehotecoder.fit_transform(df)

mode listed are Series([], Name: 0, dtype: float64)
381


In [58]:
col_non_num = [c for c in df.columns if df[c].dtype == 'object']
df.drop(columns=col_non_num, inplace=True)
col_non_num

[]

In [60]:
# fill with mode, mean, or median
df_mode, df_mean, df_median = df.mode().iloc[0], df.mean(), df.median()
df.fillna(df_median, inplace=True)
data = StandardScaler().fit_transform(df.values)

In [62]:
A_sparse = sparse.csr_matrix(data)
similarities = cosine_similarity(A_sparse)

In [64]:
similarities_sparse = cosine_similarity(A_sparse, dense_output=False)
euclidean_matrix = pd.DataFrame(squareform(pdist(data, metric='euclidean')), columns=students['id'], index=students['id'])

In [65]:
cosine_matrix = pd.DataFrame(squareform(pdist(data, metric='cosine')), columns=students['id'], index=students['id'])
cos = cosine_matrix.to_dict(orient='records')  # Here's our added param..
db.cosine_matrix.insert_many(cos)
euc = euclidean_matrix.to_dict(orient='records2q34')  # Here's our added param..
db.euclidean_matrix.insert_many(euc)

In [68]:
list(df.columns)

['gender_1',
 'gender_2',
 'InUS_1',
 'InUS_2',
 'ethnicity_1',
 'ethnicity_2',
 'ethnicity_3',
 'ethnicity_4',
 'ethnicity_5',
 'ethnicity_6',
 'ethnicity_7',
 'Usstate_1',
 'Usstate_2',
 'Usstate_3',
 'Usstate_4',
 'Usstate_5',
 'Usstate_6',
 'Usstate_7',
 'Usstate_8',
 'Usstate_9',
 'Usstate_10',
 'Usstate_11',
 'Usstate_12',
 'Usstate_13',
 'Usstate_14',
 'Usstate_15',
 'Usstate_16',
 'Usstate_17',
 'Usstate_18',
 'Usstate_19',
 'Usstate_20',
 'Usstate_21',
 'Usstate_22',
 'Usstate_23',
 'Usstate_24',
 'Usstate_25',
 'Usstate_26',
 'Usstate_27',
 'Usstate_28',
 'Usstate_29',
 'Usstate_30',
 'Usstate_31',
 'Usstate_32',
 'Usstate_33',
 'Usstate_34',
 'Usstate_35',
 'Usstate_36',
 'Usstate_37',
 'Usstate_38',
 'Usstate_39',
 'marrital_1',
 'marrital_2',
 'marrital_3',
 'marrital_4',
 'marrital_5',
 'employment_1',
 'employment_2',
 'employment_3',
 'employment_4',
 'employment_5',
 'employment_6',
 'employment_7',
 'employment_8',
 'employment_9',
 'employment_10',
 'employment_11',


In [70]:
from scipy.spatial.distance import cosine
from sklearn.metrics import jaccard_similarity_score

In [94]:
x = [1, 0, 0, 1, 0, 0]
y = [1, 0, 1, 0, 0, 0]
print(1 - cosine(x,y))
print(jaccard_similarity_score(x,y))
D = gower_distances(df)

0.5
0.6666666666666666


/home/matt/.conda/envs/weare/lib/python3.7/site-packages/ipykernel_launcher.py:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.


In [95]:
D

array([[0.        , 0.07188543, 0.05294517, ..., 0.02568973, 0.02568973,
        0.02568973],
       [0.07188543, 0.        , 0.06423889, ..., 0.04660872, 0.04660872,
        0.04660872],
       [0.05294517, 0.06423889, 0.        , ..., 0.02732194, 0.02732194,
        0.02732194],
       ...,
       [0.02568973, 0.04660872, 0.02732194, ..., 0.        , 0.        ,
        0.        ],
       [0.02568973, 0.04660872, 0.02732194, ..., 0.        , 0.        ,
        0.        ],
       [0.02568973, 0.04660872, 0.02732194, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)